In [ ]:
# Task 1
from urllib.request import urlopen
html = urlopen('https://dblp.uni-trier.de/db/conf/kdd/kdd2023.html')
html_text = bytes.decode(html.read())
with open('page.txt', 'w', encoding='UTF-8') as file:
    file.write(html_text)

In [ ]:
# Task 2
def extract(text):
    pieces = text.split('>')
    if len(pieces) == 1 and len(pieces[0].split('<')) == 1:
        return text
    if len(pieces[0].split('<')) < 2:
        pieces[0] = '<' + pieces[0]
    pieces = [piece.split('<')[0] for piece in pieces]
    output = ''.join(pieces)
    return output

track_part = html_text.split('<header class="h2">')[1:]
track_name_text = [part.split("</h2>")[0] for part in track_part]
track_name = [extract(name_text) for name_text in track_name_text]
track_name

In [ ]:
# Task 3
def sep_paper(text):
    author_info, title_info = text.split('<span class="title" itemprop="name">')
    title_info, pages_info = title_info.split('<span itemprop="pagination">')
    authors = extract(author_info)
    title = extract(title_info)
    pages = extract(pages_info).split('-')
    output = {
        "authors": authors.strip().strip(':').split(', '),
        "title": title.strip(),
        "startPage": int(pages[0]),
        "endPage": int(pages[1])
    }
    return output

track_candidate = ["Research Track Full Papers", "Applied Data Track Full Papers"]
track_obj = []

forward_authors = []  # For Task 5

for i in range(len(track_name)):
    if track_name[i] in track_candidate:
        track_info = track_part[i]
        papers_info = track_info.split('<li class="entry inproceedings"')[1:]
        papers_info = [info.split('<cite class="data tts-content" itemprop="headline">')[1] for info in papers_info]
        papers = [sep_paper(paper) for paper in papers_info]
        print(len(papers))
        track_obj.append({"track": track_name[i], "papers": papers})

        forward_authors.extend(papers_info[:10])  # For Task 5

In [ ]:
# Task 4
import json
track_str = json.dumps(track_obj, indent=2)
with open("kdd23.json", "w", encoding="UTF-8") as file:
    file.write(track_str)

In [ ]:
# Task 5
authors_info = "".join(forward_authors).split('<a href="')[1:]
authors_urls = [author.split('" itemprop="url">')[0] for author in authors_info]
authors_urls = list(set(authors_urls))
researchers_obj = []

def sep_paper_new(text, year):
    coauthor_info, title_info = text.split('<span class="title" itemprop="name">')
    title_info, *pub_info = title_info.split('</span>')
    pub_info = ''.join(pub_info).split('</cite>')[0]
    coauthors = extract(coauthor_info).strip().strip(':').split(', ')
    title = extract(title_info)
    pub = extract(pub_info).strip().strip('[content]')

    output = {
        "authors": coauthors,
        "title": title,
        "publishInfo": pub,
        "year": int(year),
    }
    return output

for i in range(len(authors_urls)):
    print(f"{i+1}/{len(authors_urls)}")

    author_url = authors_urls[i]
    author_html = urlopen(url=author_url)
    author_text = bytes.decode(author_html.read())

    author_info = author_text.split('<span class="name primary" itemprop="name">')[1]
    researcher = author_info.split('<')[0].strip('"').strip()

    try:
        orcID_info = author_info.split('<li class="orcid drop-down">')[1]
        orcID_info = orcID_info.split('</div>')[0]
        orcIDs = orcID_info.split('https://orcid.org/')[1:]
        orcID = ','.join([piece.split('">')[0] for piece in orcIDs])
    except Exception:
        orcID = ''

    papers_text = author_text.split('today</h2>')[1]
    papers_text = papers_text.split('</header>')[1]

    papers_years_info = papers_text.split('<li class="year">')[1:]
    papers_years = [year_info.split('</li>')[0] for year_info in papers_years_info]

    papers = []
    for j in range(len(papers_years)):
        papers_info = papers_years_info[j]
        papers_year = papers_years[j]
        papers_info = papers_info.split('<cite class="data tts-content" itemprop="headline">')[1:]
        papers.extend([sep_paper_new(paper, papers_year) for paper in papers_info])
    
    output_dict = {
        "researcher": researcher,
        "orcID": orcID,
        "papers": papers
    }
    researchers_obj.append(output_dict)

researchers_str = json.dumps(researchers_obj, indent=2)
with open("researchers.json", "w", encoding="UTF-8") as file:
    file.write(researchers_str)
